In [1]:
import pickle
data = pickle.load(open("./data_pik", 'rb'))

In [11]:
import pandas as pd
df = pd.DataFrame(data[400:500])

In [12]:
import random
from rouge_score import rouge_scorer
import numpy as np

def calc_rouge_scores(pred_summaries, gold_summaries, 
                                 keys=['rouge1', 'rougeL'], use_stemmer=True):
    #Calculate rouge scores
    scorer = rouge_scorer.RougeScorer(keys, use_stemmer= use_stemmer)
    
    n = len(pred_summaries)
    
    scores = [scorer.score(pred_summaries[j], gold_summaries[j]) for 
              j in range(n)] 
    
    dict_scores={}                                                            
    for key in keys:
        dict_scores.update({key: {}})
        
    
    for key in keys:
        
        precision_list = [scores[j][key][0] for j in range(len(scores))]
        recall_list = [scores[j][key][1] for j in range(len(scores))]
        f1_list = [scores[j][key][2] for j in range(len(scores))]

        precision = np.mean(precision_list)
        recall = np.mean(recall_list)
        f1 = np.mean(f1_list)
        
        dict_results = {'recall': recall, 'precision': precision, 'f1': f1}
        
        dict_scores[key] = dict_results
        
    return dict_scores

In [13]:
import spacy
import pytextrank

# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_sm")

# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank", last=True)

In [14]:
gen_gold_summaries = []

In [15]:
for i in df.iterrows():
    doc = nlp(i[1]['description'])
    doc_sum = []
    for sent in doc._.textrank.summary(limit_phrases=15, limit_sentences=5):
        doc_sum.append(str(sent))
    doc_sum = " ".join(doc_sum)
    gen_gold_summaries.append((doc_sum, i[1]['abstract']))

In [16]:
predicted = [a_tuple[0] for a_tuple in gen_gold_summaries]
gold = [a_tuple[1] for a_tuple in gen_gold_summaries]

In [18]:
calc_rouge_scores(predicted, gold, keys=['rouge1', 'rouge2', 'rougeL'])

{'rouge1': {'recall': 0.24554938040368285,
  'precision': 0.5184458944573926,
  'f1': 0.316671162480485},
 'rouge2': {'recall': 0.06772514078476005,
  'precision': 0.14590445832870358,
  'f1': 0.08802421562268091},
 'rougeL': {'recall': 0.138578344100124,
  'precision': 0.3010196985802842,
  'f1': 0.17977204297758503}}

In [31]:
gen_gold_summaries_gensim = []

In [32]:
from gensim.summarization import summarize

In [33]:
for i in df.iterrows():
    doc_sum = summarize(text, ratio=0.05)
    gen_gold_summaries.append((doc_sum, i[1]['abstract']))

In [34]:
predicted = [a_tuple[0] for a_tuple in gen_gold_summaries]
gold = [a_tuple[1] for a_tuple in gen_gold_summaries]
calc_rouge_scores(predicted, gold, keys=['rouge1', 'rouge2', 'rougeL'])

{'rouge1': {'recall': 0.20933957569802464,
  'precision': 0.3772865020506188,
  'f1': 0.2540093808526198},
 'rouge2': {'recall': 0.04732410885391849,
  'precision': 0.08960990974332474,
  'f1': 0.05837365825262688},
 'rougeL': {'recall': 0.1293273399889933,
  'precision': 0.2341377656976672,
  'f1': 0.15674908313029465}}